In [7]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os, sys, time
import argparse
import tarfile
import re

##impor  BMD files from directory
##TODO: combine bMD processing to single file, combine LPR processing to single file
import bmd_analysis_morpho as bmd
import bmd_analysis_LPR_7_PAH_t0_t239 as bmd_LPR
import format_LPR_input as format_LPR
import format_morpho_input as format_morpho

/Users/degn400/Desktop/Git_Repos/srpAnalytics/zfBmd
/Users/degn400/Desktop/Git_Repos/srpAnalytics/zfBmd


# Journey that a single morpho file takes

In [24]:
mfiles = './test_files/7_PAH_zf_morphology_data_2020NOV11_tall_3756.csv'

morpho_input_csv_file_name_wide = mfiles.split("/")[-1] + "_wide_DNC_0_"+ "fd" + ".csv"

In [55]:
df_morph = pd.read_csv(mfiles, header = 0)

# Keep onloy relevant columns
columns_to_keep = ['chemical.id', 'conc', 'plate.id', 'well', 'endpoint', 'value']
df_morph_select = df_morph.loc[:,columns_to_keep]

# Convert numerics to strings 
df_morph_select['plate.id'] = df_morph_select['plate.id'].astype(str)

In [57]:
df_morph_select.dtypes

chemical.id      int64
conc           float64
plate.id        object
well            object
endpoint        object
value          float64
dtype: object

In [68]:
df_reformatted = pd.DataFrame()

for chemical_index in pd.unique(df_morph['chemical.id']):

    print("chemical_index:" + str(chemical_index))
    #total_number_of_unique_chemicals += 1
    morph_data_chemical = df_morph_select.loc[df_morph['chemical.id'] == chemical_index,:]

    # Append chemical_plate_well as a unique identifier
    morph_data_chemical.insert(0, 'chemical_plate_well', morph_data_chemical.loc[:,['chemical.id','plate.id', 'well']].apply(lambda x: '_'.join(x.map(str)), axis = 1))

    for cpw in np.unique(morph_data_chemical.chemical_plate_well):
        #total_number_of_chemical_plate_well += 1
        temp_df = morph_data_chemical.loc[morph_data_chemical.chemical_plate_well == cpw,:]

        temp_df_grouped = temp_df.groupby(['chemical.id', 'plate.id', 'well'])
        for name, group in temp_df_grouped:

            ## JUSTIFICATION: 7 PAH dataset doesn't have "BRAI" endpoint.
            ## On the other hand, extracts/phase I,II have "BRAI" endpoint.
            if 'BRAI' not in temp_df.endpoint.values: # as 7 PAH
                try:
                    #            if(len(group.endpoint) == 14):
                    temp = pd.DataFrame( {
                        'chemical.id': pd.unique(temp_df['chemical.id']),
                        'plate.id': pd.unique(temp_df['plate.id']),
                        'well': pd.unique(temp_df['well']),
                        'chemical_plate_well': pd.unique(temp_df['chemical_plate_well']),
                        'conc': pd.unique(temp_df['conc']),
                        'AXIS': temp_df.value[temp_df.endpoint == 'AXIS'].values,
                        'BRN_': temp_df.value[temp_df.endpoint == 'BRN_'].values,
                        'CRAN': temp_df.value[temp_df.endpoint == 'CRAN'].values,
                        'DNC_': temp_df.value[temp_df.endpoint == 'DNC_'].values,
                        'DP24': temp_df.value[temp_df.endpoint == 'DP24'].values,
                        'EDEM': temp_df.value[temp_df.endpoint == 'EDEM'].values,
                        'LTRK': temp_df.value[temp_df.endpoint == 'LTRK'].values,
                        'MO24': temp_df.value[temp_df.endpoint == 'MO24'].values,
                        'MORT': temp_df.value[temp_df.endpoint == 'MORT'].values,
                        'MUSC': temp_df.value[temp_df.endpoint == 'MUSC'].values,
                        'NC__': temp_df.value[temp_df.endpoint == 'NC__'].values,
                        'SKIN': temp_df.value[temp_df.endpoint == 'SKIN'].values,
                        'SM24': temp_df.value[temp_df.endpoint == 'SM24'].values,
                        'TCHR': temp_df.value[temp_df.endpoint == 'TCHR'].values,
                    }  )
                    df_reformatted = pd.concat([df_reformatted, temp])
                except:
                    print ("len(group.endpoint) != 14")
                    print ("chemical_plate_well:" + str(cpw))
            else: #as extracts
                temp = pd.DataFrame(
                    {
                    'chemical.id': pd.unique(temp_df['chemical.id']),
                    'plate.id': pd.unique(temp_df['plate.id']),
                    'well': pd.unique(temp_df['well']),
                    'chemical_plate_well': pd.unique(temp_df['chemical_plate_well']),
                    'conc': pd.unique(temp_df['conc']),
                    'AXIS': temp_df.value[temp_df.endpoint == 'AXIS'].values,
                    'BRAI': temp_df.value[temp_df.endpoint == 'BRAI'].values,
                    'CFIN': temp_df.value[temp_df.endpoint == 'CFIN'].values,
                    'CIRC': temp_df.value[temp_df.endpoint == 'CIRC'].values,
                    'DNC_': temp_df.value[temp_df.endpoint == 'DNC_'].values,
                    'DP24': temp_df.value[temp_df.endpoint == 'DP24'].values,
                    'EYE_': temp_df.value[temp_df.endpoint == 'EYE_'].values,
                    'JAW_': temp_df.value[temp_df.endpoint == 'JAW_'].values,
                    'MO24': temp_df.value[temp_df.endpoint == 'MO24'].values,
                    'MORT': temp_df.value[temp_df.endpoint == 'MORT'].values,
                    'NC24': temp_df.value[temp_df.endpoint == 'NC24'].values,
                    'NC__': temp_df.value[temp_df.endpoint == 'NC__'].values,
                    'OTIC': temp_df.value[temp_df.endpoint == 'OTIC'].values,
                    'PE__': temp_df.value[temp_df.endpoint == 'PE__'].values,
                    'PFIN': temp_df.value[temp_df.endpoint == 'PFIN'].values,
                    'PIG_': temp_df.value[temp_df.endpoint == 'PIG_'].values,
                    'SM24': temp_df.value[temp_df.endpoint == 'SM24'].values,
                    'SNOU': temp_df.value[temp_df.endpoint == 'SNOU'].values,
                    'SOMI': temp_df.value[temp_df.endpoint == 'SOMI'].values,
                    'SWIM': temp_df.value[temp_df.endpoint == 'SWIM'].values,
                    'TRUN': temp_df.value[temp_df.endpoint == 'TRUN'].values,
                    'TR__': temp_df.value[temp_df.endpoint == 'TR__'].values,
                    'YSE_': temp_df.value[temp_df.endpoint == 'YSE_'].values,
                    })
                df_reformatted = pd.concat([df_reformatted, temp])
#print ("total_number_of_unique_chemicals:" + str(total_number_of_unique_chemicals))
#print ("total_number_of_chemical_plate_well:" + str(total_number_of_chemical_plate_well))
#end_time = time.time()
#time_took = str(round((end_time-start_time), 1)) + " seconds"
#print ("Done, it took:"+str(time_took))

pd.set_option('display.max_columns', None)
print (df_reformatted.head())
#print ("df_reformatted.shape:" + str(df_reformatted.shape)) #(288, 19)

df_reformatted_DNC_0 = pd.DataFrame()

df_reformatted_DNC_0 = df_reformatted.loc[df_reformatted['DNC_'] == 0.0]
print ("df_reformatted_DNC_0.shape:" + str(df_reformatted_DNC_0.shape)) #(287, 19)

df_reformatted_DNC_0.to_csv("./Reformatted.csv", index=False)

chemical_index:3756
   chemical.id plate.id well chemical_plate_well   conc  AXIS  BRN_  CRAN  \
0         3756    20544  A01      3756_20544_A01  100.0   NaN   NaN   NaN   
0         3756    20544  A02      3756_20544_A02  100.0   0.0   0.0   0.0   
0         3756    20544  A03      3756_20544_A03  100.0   NaN   NaN   NaN   
0         3756    20544  A04      3756_20544_A04  100.0   0.0   0.0   0.0   
0         3756    20544  A05      3756_20544_A05  100.0   0.0   0.0   0.0   

   DNC_  DP24  EDEM  LTRK  MO24  MORT  MUSC  NC__  SKIN  SM24  TCHR  
0   0.0   NaN   NaN   NaN   1.0   NaN   NaN   NaN   NaN   NaN   NaN  
0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  
0   0.0   NaN   NaN   NaN   1.0   NaN   NaN   NaN   NaN   NaN   NaN  
0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  
0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  
df_reformatted_DNC_0.shape:(287, 19)


In [89]:
pivot_wider = df_morph_select.pivot_table(index = ['chemical.id', 'plate.id', 'well', 'conc'], columns = ['endpoint'], 
                                           values = ['value'], aggfunc = 'first')
pivot_wider.reset_index()

chemical.id plate.id well   conc value                                \
endpoint                                   AXIS BRN_ CRAN DNC_ DP24 EDEM LTRK   
0               3756    20544  A01  100.0   NaN  NaN  NaN  0.0  NaN  NaN  NaN   
1               3756    20544  A02  100.0   0.0  0.0  0.0  0.0  0.0  0.0  0.0   
2               3756    20544  A03  100.0   NaN  NaN  NaN  0.0  NaN  NaN  NaN   
3               3756    20544  A04  100.0   0.0  0.0  0.0  0.0  0.0  0.0  0.0   
4               3756    20544  A05  100.0   0.0  0.0  0.0  0.0  0.0  0.0  0.0   
..               ...      ...  ...    ...   ...  ...  ...  ...  ...  ...  ...   
283             3756    20625  H08    0.0   0.0  0.0  0.0  0.0  0.0  0.0  0.0   
284             3756    20625  H09    0.0   0.0  0.0  0.0  0.0  0.0  0.0  0.0   
285             3756    20625  H10    0.0   0.0  0.0  0.0  0.0  0.0  0.0  0.0   
286             3756    20625  H11    0.0   0.0  0.0  0.0  0.0  0.0  0.0  0.0   
287             3756    20625  H12    0.0   0.0  0.0  0.0  0.0  0.0  0.0  0.0   

                                             
endpoint MO24 MORT MUSC NC__ SKIN SM24 TCHR  
0         1.0  NaN  NaN  NaN  NaN  NaN  NaN  
1         0.0  0.0  0.0  0.0  0.0  0.0  0.0  
2         1.0  NaN  NaN  NaN  NaN  NaN  NaN  
3         0.0  0.0  0.0  0.0  0.0  0.0  0.0  
4         0.0  0.0  0.0  0.0  0.0  0.0  0.0  
..        ...  ...  ...  ...  ...  ...  ...  
283       0.0  0.0  0.0  0.0  0.0  0.0  0.0  
284       0.0  0.0  0.0  0.0  0.0  0.0  0.0  
285       0.0  0.0  0.0  0.0  0.0  0.0  0.0  
286       0.0  0.0  0.0  0.0  0.0  0.0  0.0  
287       0.0  0.0  0.0  0.0  0.0  0.0  0.0  

[288 rows x 18 columns]